# Intro to LLMstudio
In this tutorial you'll be running the proxy and the tracker so you can automatically log the usage metrics and costs in the default database (`llmstudio_mgmt.db`).

You'll learn:
1. How to connect to any provider available (VertexAI, OpenAI, etc.)
2. Make sync and async calls both with and without streaming
3. See the saved logs

First things first:
* run `pip install llmstudio[proxy, tracker]`
* update your .env file with `GOOGLE_API_KEY` or `OPENAI_API_KEY`


## Start proxy and tracking servers
This step is to set the proxy and tracking servers.
You can opt out to start such servers (see `llmstudio-core` for the lightest version)

In [1]:
from pprint import pprint
from llmstudio.server import start_servers
from llmstudio_tracker.tracker import TrackingConfig
from llmstudio_proxy.provider import ProxyConfig

# start servers
start_servers(proxy=True, tracker=True)

proxy = ProxyConfig(host="0.0.0.0", port="8001")  # default port is 50001. set the environment varible to specify which host and port; LLMSTUDIO_ENGINE_HOST, LLMSTUDIO_ENGINE_PORT
tracking = TrackingConfig(host="0.0.0.0", port="8002")  # default port is 50002. set the environment varible to specify which host and port; LLMSTUDIO_TRACKING_HOST, LLMSTUDIO_TRACKING_PORT




Running LLMstudio Proxy on http://0.0.0.0:8001 
Running LLMstudio Tracking on http://0.0.0.0:8002 


In [2]:
from llmstudio.providers import LLM

vertexai = LLM("vertexai",
               proxy_config=proxy,
               tracking_config=tracking,
               session_id="vertexai-1")

openai = LLM("openai",
             proxy_config=proxy,
             tracking_config=tracking,
             session_id="openai-1")


Connected to LLMStudio Proxy @ 0.0.0.0:8001
Connected to LLMStudio Proxy @ 0.0.0.0:8001


# VertexAI
### Chat (non-stream)

In [3]:
response = vertexai.chat("What's your name", model="gemini-1.5-pro-latest")
print(response.chat_output)
pprint(response.metrics)

I am a large language model, trained by Google. I don't have a name like a person does.  How can I help you today? 😊 

{'cost_usd': 0.00034789999999999995,
 'input_tokens': 4,
 'inter_token_latency_s': 0.08976759229387556,
 'latency_s': 1.7716200351715088,
 'output_tokens': 33,
 'time_to_first_token_s': 1.1004998683929443,
 'tokens_per_second': 4.515640962044961,
 'total_tokens': 37}


### Chat Async (non-stream)

In [4]:
await vertexai.achat("What's your name", model="gemini-1.5-pro-latest")

ChatCompletion(id='e9e21b7c-5e05-446d-b1d9-fccd82df6fd0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I am a large language model, so I don't have a name like a person. You can call me your helpful assistant! 😊  How can I help you today? \n", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1729767489, model='gemini-1.5-pro-latest', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None, chat_input="What's your name", chat_output="I am a large language model, so I don't have a name like a person. You can call me your helpful assistant! 😊  How can I help you today? \n", chat_output_stream='', context=[{'role': 'user', 'content': "What's your name"}], provider='vertexai', deployment='gemini-1.5-pro-latest', timestamp=1729767489.438024, parameters={}, metrics={'input_tokens': 4, 'output_tokens': 37, 'total_tokens': 41, 'cost_usd': 0.00038989999999999994, 'latency_s

### Chat (stream)

In [5]:
response = vertexai.chat("Write a paragfraph about space", model="gemini-1.5-pro-latest", is_stream=True)
for i, chunk in enumerate(response):
    if i%2==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




Space, a vast and enigmatic expanse, beckons us with its celestial wonders.  

Twinkling stars, like distant diamonds scattered across a velvet canvas, illuminate the cosmic tapestry.  Planets, both majestic and mysterious, dance in intricate orbits around their parent stars.  Nebulas, vast clouds of gas and dust, paint the

 cosmos with vibrant hues of crimson, emerald, and azure.  Black holes, enigmatic and awe-inspiring, warp the fabric of spacetime, challenging our understanding of the universe.  From the smallest particles to the largest galaxies, space is a realm of endless discovery, igniting our curiosity and reminding us of the infinite beauty that lies beyond our world. 




## Metrics:
{'cost_usd': 0.0015148,
 'input_tokens': 8,
 'inter_token_latency_s': 0.2275336438959295,
 'latency_s': 3.6205759048461914,
 'output_tokens': 144,
 'time_to_first_token_s': 1.0920016765594482,
 'tokens_per_second': 3.3143898416651982,
 'total_tokens': 152}


### Chat Async (stream)

In [6]:
i=0
async for chunk in await vertexai.achat("Write a paragfraph about space", model="gemini-1.5-pro-latest", is_stream=True):
    if i%20==2:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1


Space, a breathtaking tapestry of darkness sprinkled with celestial wonders, continues to inspire awe and

 ignite our curiosity.  Vast distances stretch between twinkling stars, each a fiery furnace birthing the elements of life itself. Swirling galaxies, majestic and ancient, hold secrets yet to be unraveled.  From the icy rings of Saturn to the crimson storms of Jupiter, space whispers tales of cosmic creation and invites us to explore the boundless possibilities beyond our pale blue dot. 


## Metrics:
{'cost_usd': 0.0010213,
 'input_tokens': 8,
 'inter_token_latency_s': 0.19331757227579752,
 'latency_s': 2.481858015060425,
 'output_tokens': 97,
 'time_to_first_token_s': 0.7149538993835449,
 'tokens_per_second': 4.029239359914203,
 'total_tokens': 105}


### Get session metrics and usage

In [7]:
from llmstudio_tracker.tracker import Tracker

tracker = Tracker(tracking_config=tracking)

In [8]:
logs = tracker.get_session_logs(session_id="vertexai-1")
logs.json()[-1]


{'chat_input': 'Write a paragfraph about space',
 'chat_output': 'Space, a breathtaking tapestry of darkness sprinkled with celestial wonders, continues to inspire awe and ignite our curiosity.  Vast distances stretch between twinkling stars, each a fiery furnace birthing the elements of life itself. Swirling galaxies, majestic and ancient, hold secrets yet to be unraveled.  From the icy rings of Saturn to the crimson storms of Jupiter, space whispers tales of cosmic creation and invites us to explore the boundless possibilities beyond our pale blue dot. \n',
 'session_id': 'vertexai-1',
 'context': [{'role': 'user', 'content': 'Write a paragfraph about space'}],
 'provider': 'vertexai',
 'model': 'gemini-1.5-pro-latest',
 'deployment': 'gemini-1.5-pro-latest',
 'parameters': {},
 'metrics': {'input_tokens': 8,
  'output_tokens': 97,
  'total_tokens': 105,
  'cost_usd': 0.0010213,
  'latency_s': 2.481858015060425,
  'time_to_first_token_s': 0.7149538993835449,
  'inter_token_latency_s'

# OpenAI
### Chat (non-stream)

In [9]:
response = openai.chat("What's your name", model="gpt-4o")
print(response.chat_output)
pprint(response.metrics)

I’m an AI language model created by OpenAI, and I don’t have a personal name. You can call me Assistant if you’d like!
{'cost_usd': 0.00047,
 'input_tokens': 4,
 'inter_token_latency_s': 0.011531452978810957,
 'latency_s': 0.9431579113006592,
 'output_tokens': 30,
 'time_to_first_token_s': 0.5848979949951172,
 'tokens_per_second': 33.92857083271506,
 'total_tokens': 34}


### Chat Async (non-stream)

In [10]:
await openai.achat("What's your name", model="gpt-4o")

ChatCompletion(id='df697714-885b-4eb1-87fd-3ad798a33205', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I’m called Assistant. How can I help you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1729767513, model='gpt-4o', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None, chat_input="What's your name", chat_output='I’m called Assistant. How can I help you today?', chat_output_stream='', context=[{'role': 'user', 'content': "What's your name"}], provider='openai', deployment='gpt-4o-2024-08-06', timestamp=1729767513.969525, parameters={}, metrics={'input_tokens': 4, 'output_tokens': 12, 'total_tokens': 16, 'cost_usd': 0.0002, 'latency_s': 0.8102009296417236, 'time_to_first_token_s': 0.70477294921875, 'inter_token_latency_s': 0.008057924417349009, 'tokens_per_second': 17.279664201559108})

### Chat (stream)

In [11]:
response = openai.chat("Write a paragfraph about space", model="gpt-4o", is_stream=True)
for i, chunk in enumerate(response):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




Space, the vast expanse that stretches beyond the confines of our planet, is a realm of

 endless mystery and wonder. Encompassing everything from our solar system to distant galaxies, it is a

 frontier that has captivated human imagination for centuries. In the vacuum of space, celestial bodies such as stars

, planets, and asteroids dance to the gravitational rhythms dictated by the massive forces at play. The

 study of space, through the field of astronomy and the innovation of space exploration, has led to groundbreaking

 discoveries about the origins of the universe and the fundamental laws of physics. With advancements in technology, humanity

 has been able to send probes, satellites, and telescopes into this inky void, gaining unprecedented

 insights into phenomena such as black holes and cosmic microwave background radiation. As we continue to explore and unravel

 the mysteries of space, we broaden our understanding not just of the cosmos, but of our place within

 it, 

### Chat Async (stream)

In [12]:
i=0
async for chunk in await openai.achat("Write a paragfraph about space", model="gpt-4o-mini", is_stream=True):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1




Space, the vast and enigmatic expanse beyond our planet, has fascinated humanity for centuries. It

 is a realm of unparalleled beauty and mystery, filled with shimmering stars, swirling galaxies, and the haunting

 silence of the cosmos. Stretching infinitely in all directions, space encompasses everything from our Moon and the

 planets of our solar system to distant quasars and black holes, each holding secrets of the universe's

 origin and fate. As we continue to explore this infinite void through advanced telescopes and space missions,

 we uncover not only the mechanics of celestial bodies but also the fundamental questions of existence. From the allure

 of potential extraterrestrial life to the awe-inspiring phenomena of dark matter and dark energy, space challenges

 our understanding and ignites our imagination, inviting us to ponder our place in the universe.

## Metrics:
{'cost_usd': 9.779999999999999e-05,
 'input_tokens': 8,
 'inter_token_latency_s': 0.01646837403502645

In [13]:
from llmstudio_tracker.tracker import Tracker

tracker = Tracker(tracking_config=tracking)

In [14]:
logs = tracker.get_session_logs(session_id="openai-1")
logs.json()[-1]

{'chat_input': 'Write a paragfraph about space',
 'chat_output': "Space, the vast and enigmatic expanse beyond our planet, has fascinated humanity for centuries. It is a realm of unparalleled beauty and mystery, filled with shimmering stars, swirling galaxies, and the haunting silence of the cosmos. Stretching infinitely in all directions, space encompasses everything from our Moon and the planets of our solar system to distant quasars and black holes, each holding secrets of the universe's origin and fate. As we continue to explore this infinite void through advanced telescopes and space missions, we uncover not only the mechanics of celestial bodies but also the fundamental questions of existence. From the allure of potential extraterrestrial life to the awe-inspiring phenomena of dark matter and dark energy, space challenges our understanding and ignites our imagination, inviting us to ponder our place in the universe.",
 'session_id': 'openai-1',
 'context': [{'role': 'user', 'cont